![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [70]:
import pandas as pd
import numpy as np

# Start coding here...

In [71]:
df = pd.read_csv("bank_marketing.csv")

for col in ["credit_default", "mortgage", "previous_outcome", "campaign_outcome"]:
    print(col)
    print("--------------")
    print(df[col].value_counts())

credit_default
--------------
no         32588
unknown     8597
yes            3
Name: credit_default, dtype: int64
mortgage
--------------
yes        21576
no         18622
unknown      990
Name: mortgage, dtype: int64
previous_outcome
--------------
nonexistent    35563
failure         4252
success         1373
Name: previous_outcome, dtype: int64
campaign_outcome
--------------
no     36548
yes     4640
Name: campaign_outcome, dtype: int64


In [72]:
df.head()

,client_id,age,job,marital,education,credit_default,mortgage,month,day,contact_duration,number_contacts,previous_campaign_contacts,previous_outcome,cons_price_idx,euribor_three_months,campaign_outcome
0,0,56,housemaid,married,basic.4y,no,no,may,13,261,1,0,nonexistent,93.994,4.857,no
1,1,57,services,married,high.school,unknown,no,may,19,149,1,0,nonexistent,93.994,4.857,no
2,2,37,services,married,high.school,no,yes,may,23,226,1,0,nonexistent,93.994,4.857,no
3,3,40,admin.,married,basic.6y,no,no,may,27,151,1,0,nonexistent,93.994,4.857,no
4,4,56,services,married,high.school,no,no,may,3,307,1,0,nonexistent,93.994,4.857,no


In [73]:
# init client Data Frame
client_df = pd.DataFrame({
    "client_id": df["client_id"],
    "age": df["age"]
})

# Remove Index
client_df = client_df.reset_index(drop=True)

client_df.head()

,client_id,age
0,0,56
1,1,57
2,2,37
3,3,40
4,4,56


In [74]:
# Clean job col and add to client_df
job = df["job"].str.replace('.', '_', regex=False) 
client_df["job"] = job

client_df.head()

,client_id,age,job
0,0,56,housemaid
1,1,57,services
2,2,37,services
3,3,40,admin_
4,4,56,services


In [75]:
# No cleaning needed for material 
client_df["marital"] = df["marital"]

client_df.head()

,client_id,age,job,marital
0,0,56,housemaid,married
1,1,57,services,married
2,2,37,services,married
3,3,40,admin_,married
4,4,56,services,married


In [76]:
# Clean education col and add to client_df
education = df["education"].str.replace('.', '_', regex=False).replace('unknown', np.NaN, regex=False)
client_df["education"] = education

client_df.head()

,client_id,age,job,marital,education
0,0,56,housemaid,married,basic_4y
1,1,57,services,married,high_school
2,2,37,services,married,high_school
3,3,40,admin_,married,basic_6y
4,4,56,services,married,high_school


In [77]:
# Clean credit_default col and add to client_df
credit_default = df["credit_default"].apply(lambda x: True if x == "yes" else False)
client_df["credit_default"] = credit_default

client_df.head()

,client_id,age,job,marital,education,credit_default
0,0,56,housemaid,married,basic_4y,False
1,1,57,services,married,high_school,False
2,2,37,services,married,high_school,False
3,3,40,admin_,married,basic_6y,False
4,4,56,services,married,high_school,False


In [78]:
# Clean mortgage col and add to client_df
mortgage = df["mortgage"].apply(lambda x: True if x == "yes" else False)
client_df["mortgage"] = mortgage

client_df.head()

,client_id,age,job,marital,education,credit_default,mortgage
0,0,56,housemaid,married,basic_4y,False,False
1,1,57,services,married,high_school,False,False
2,2,37,services,married,high_school,False,True
3,3,40,admin_,married,basic_6y,False,False
4,4,56,services,married,high_school,False,False


In [79]:
# Client Data Is Now Done, we can save as client.csv
client_df.to_csv('client.csv', index=False)

In [80]:
# Init campaign df and populate columns that do not need to be cleaned, as well as simple yes/no t/f 
campaign_df = pd.DataFrame({
    "client_id": df["client_id"],
    "number_contacts": df["number_contacts"],
    "contact_duration": df["contact_duration"],
    "previous_campaign_contacts": df["previous_campaign_contacts"],
    "previous_outcome": df["previous_outcome"].apply(lambda x: True if x == "success" else False),
    "campaign_outcome": df["campaign_outcome"].apply(lambda x: True if x == "yes" else False)
})

campaign_df.head()

,client_id,number_contacts,contact_duration,previous_campaign_contacts,previous_outcome,campaign_outcome
0,0,1,261,0,False,False
1,1,1,149,0,False,False
2,2,1,226,0,False,False
3,3,1,151,0,False,False
4,4,1,307,0,False,False


In [81]:
# Clean last_contact_date col and change format to "YYYY-MM-DD" 

campaign_df["last_contact_date"] = pd.to_datetime('2022 ' + df['month'] + ' ' + df['day'].astype(str))

# Format as YYYY-MM-DD
campaign_df["last_contact_date"] = campaign_df["last_contact_date"].dt.strftime('%Y-%m-%d')

campaign_df.head()

,client_id,number_contacts,contact_duration,previous_campaign_contacts,previous_outcome,campaign_outcome,last_contact_date
0,0,1,261,0,False,False,2022-05-13
1,1,1,149,0,False,False,2022-05-19
2,2,1,226,0,False,False,2022-05-23
3,3,1,151,0,False,False,2022-05-27
4,4,1,307,0,False,False,2022-05-03


In [82]:
# Campaign Data Is Now Done, we can save as campaign.csv
campaign_df.to_csv('campaign.csv', index=False)

In [83]:
# Init the economics table and populate it with relevant data. Nothing needs to be cleaned
economics_df = pd.DataFrame({
    "client_id": df["client_id"],
    "cons_price_idx": df["cons_price_idx"],
    "euribor_three_months": df["euribor_three_months"]
})

economics_df.head()

,client_id,cons_price_idx,euribor_three_months
0,0,93.994,4.857
1,1,93.994,4.857
2,2,93.994,4.857
3,3,93.994,4.857
4,4,93.994,4.857


In [84]:
# Econ data Is Now Done, we can save as economics.csv
economics_df.to_csv("economics.csv", index=False)